# [Omnisci: "Vega at a Glance"](https://docs.omnisci.com/latest/6_VegaAtaGlance.html)

![](https://docs.omnisci.com/latest/images/6_demoScreenshot.png)

In [ ]:
import altair as alt
import ibis_vega_transform
import ibis

conn = ibis.mapd.connect(
    host='metis.mapd.com', user='mapd', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("tweets_nov_feb")
x, y = t.goog_x, t.goog_y

In [ ]:
WIDTH = 385
HEIGHT = 564
X_DOMAIN = [
        -3650484.1235206556,
        7413325.514451755
      ]
Y_DOMAIN = [
        -5778161.9183506705,
        10471808.487466192
      ]

In [ ]:
DEBOUNCE_MS = 100

# ibis_vega_transform.altair.FALLBACK = False

scales = alt.selection_interval(
    bind='scales',
    on=f'[mousedown, window:mouseup] > window:mousemove!{{0, {DEBOUNCE_MS}}}',
    translate=f'[mousedown, window:mouseup] > window:mousemove!{{0, {DEBOUNCE_MS}}}',
    zoom=f'wheel!{{0, {DEBOUNCE_MS}}}',
)

alt.Chart(t[x, y], width=WIDTH, height=HEIGHT).mark_rect().encode(
    alt.X(
        'bin_x:Q',
        bin=alt.Bin(binned=True),
        title='goog_x',
        scale=alt.Scale(domain=X_DOMAIN)
    ),
    alt.X2('bin_x_end'),
    alt.Y(
        'bin_y:Q',
        bin=alt.Bin(binned=True),
        title='goog_y',
        scale=alt.Scale(domain=Y_DOMAIN)
    ),
    alt.Y2('bin_y_end'),
    tooltip='count()'
).add_selection(
    scales
).transform_filter(
    scales
).transform_bin(
    'bin_x',
    'goog_x',
    bin=alt.Bin(maxbins=WIDTH)
).transform_bin(
    'bin_y',
    'goog_y',
    bin=alt.Bin(maxbins=HEIGHT)
)

In [ ]:
# NOTES

# print(t[
#     t.goog_x.histogram(
#         base=ibis.literal(X_DOMAIN[0], 'float64').cast('float32'),
#         binwidth=ibis.literal((X_DOMAIN[1] -  X_DOMAIN[0]) / WIDTH, 'float64').cast('float32')
#     ).name("x_bin"),
#     t.goog_y.histogram(
#         base=ibis.literal(Y_DOMAIN[0], 'float64').cast('float32'),
#         binwidth=ibis.literal((Y_DOMAIN[1] -  Y_DOMAIN[0]) / HEIGHT, 'float64').cast('float32'),
        
#     ).name("y_bin"),
# ].group_by([
#     "x_bin",
#     "y_bin"
# ]).aggregate(
#     t.count()
# ).compile())


# def bin(transform, expr: ibis.Expr) -> ibis.Expr:
#     """
#     Apply a vega bin transform to an ibis expression.
#     https://vega.github.io/vega/docs/transforms/bin/
#     """

#     field = expr[transform["field"]]
#     as_left, as_right = transform["as"]
#     maxbins = transform["maxbins"]
#     extent = expr[transform["extent"]]

    
#     # Case these to floats to work around
#     # https://github.com/ibis-project/ibis/issues/1934
#     binwidth = ((t_max - t_min) / maxbins).name("bin_width")

#     bin_ = ((field - t_min) / binwidth).floor()
#     left = (t_min + (bin_ * binwidth)).name(as_left)
#     right = (left + binwidth).name(as_right)

#     # add the two new fields and remove the initial column
#     return expr.select([left, right] + [c for c in expr.columns if c != transform["field"]])



# print(bin({"field": "goog_x", "as": ["bin_x", "bin_x_end"], "maxbins": WIDTH, "extent": "goog_x"}, t[t['goog_x']]).groupby(["bin_x", "bin_x_end"]).aggregate(ibis.expr.api.count(t)).compile())


# t_new = t[t.goog_x.histogram(
#     base=ibis.literal(X_DOMAIN[0], 'float64').cast('float32'),
#     binwidth=ibis.literal((X_DOMAIN[1] -  X_DOMAIN[0]) / WIDTH, 'float64').cast('float32')
# ).name('hist')]
# print(t_new.compile())